In [11]:
import sys
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/home/hbkoziel/awi-models/codes/modules/modules')

import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from load_mesh_data_new import *
import time
from netCDF4 import Dataset
import sys
from DefSec import DefSec
from vec_rotate_r2g import vec_rotate_r2g
import pickle

runid='Arc12'
meshpath='/home/hbkoziel/awi-models/data/mesh/meshArc4.5/'
datapath='/home/hbkoziel/awi-models/figures/'+runid+'/'
resultpath = '/scratch/usr/hbkvsk12/hlrn3_work2/results/'+runid+'/Oldfiles/'

din_id, dsi_id, u_id, v_id = 'tr01', 'tr18', 'u', 'v'

first_year = 2000	
last_year  = 2000
years      = np.arange(first_year,last_year+1,1)

alpha, beta, gamma=[50, 15, -90]

almost_black = [(38.,38.,38.)]
r,g,b = almost_black[0]
almost_black[0]=(r / 255., g / 255., b / 255.)
ab = almost_black[0]

xstart = np.array([-169.8, -62.0, -20.5, 20.0])
ystart = np.array([  66.1,  66.5,  79.0, 78.5])
xend   = np.array([-167.8, -50.0,  18.0, 20.1])
yend   = np.array([  65.7,  66.5,  79.0, 73.6])
pos    = np.array([xstart,ystart,xend,yend])

reso = 10
num_rec = 175
num_section=4

print ('Loading mesh')
t0 = time.clock()
mesh = load_mesh(meshpath)
print time.clock() - t0, "seconds process time"
print ('Starting calculations')

n1_all,n2_all,intcoef_all,vol_el_all,k_all,m_all,nvec_all=DefSec(num_section,mesh,pos)      

print 'start to read and analyze data ... '  
transp    = np.zeros(shape=(len(years)*num_rec,3,num_section))
transpdin = np.zeros(shape=(len(years)*num_rec,3,num_section))
transpdsi = np.zeros(shape=(len(years)*num_rec,3,num_section))  

depth = -mesh.zlevs
#depth = depth[::-1]  

##########
cnt = 0
for year in years:
  ncfile = resultpath+runid+'.'+str(year)+'.oce.mean.nc'
  #ncfile = resultpath+runid+'.'+str(years[ind])+'.oce.mean.nc'
  f = Dataset(ncfile, 'r')
  u3d       = f.variables[u_id][:]
  v3d       = f.variables[v_id][:]
  din3d     = f.variables[din_id][:]
  dsi3d     = f.variables[dsi_id][:]
  for index in range(0,num_rec):
    print year, index    
    vy = v3d[index,:]
    vx = u3d[index,:]
    dinm = din3d[index,:]
    dsim = dsi3d[index,:]
        
    (um, vm)    = vec_rotate_r2g(mesh.x3, mesh.y3, vx, vy);
           
    for nsec in range(0,num_section): 
      print 'Section',nsec
      k       = k_all[nsec]
      m       = m_all[nsec]
      n1      = n1_all[0:k,0:m,nsec]
      n2      = n2_all[0:k,0:m,nsec]
      intcoef = intcoef_all[0:k,0:m,nsec]
      nvec    = nvec_all[:,nsec]
      vol_el  = vol_el_all[0:k-1,0:m-1,nsec]

      x3s  = mesh.x3[n1]*(1-intcoef) + mesh.x3[n2]*intcoef
      y3s  = mesh.y3[n1]*(1-intcoef) + mesh.y3[n2]*intcoef
      us   = um[n1]*(1-intcoef) + um[n2]*intcoef
      vs   = vm[n1]*(1-intcoef) + vm[n2]*intcoef
      dins = dinm[n1]*(1-intcoef) + dinm[n2]*intcoef 
      dsis = dsim[n1]*(1-intcoef) + dsim[n2]*intcoef    
      vels = us*nvec[0] + vs*nvec[1]
      
      x_el1   = x3s[:,0:len(x3s[0,:])-1] + x3s[:,1:len(x3s[0,:])]
      x_el    = (x_el1[0:len(x3s[:,0])-1] + x_el1[1:len(x3s[0,:]),:])/4.0
      y_el1   = y3s[:,0:len(y3s[0,:])-1] + y3s[:,1:len(y3s[0,:])]
      y_el    = (y_el1[0:len(y3s[:,0])-1] + y_el1[1:len(y3s[0,:]),:])/4.0
      din_el1 = dins[:,0:len(dins[0,:])-1] + dins[:,1:len(dins[0,:])]
      din_el  = (din_el1[0:len(dins[:,0])-1] + din_el1[1:len(dins[0,:]),:])/4.0
      dsi_el1 = dsis[:,0:len(dsis[0,:])-1] + dsis[:,1:len(dsis[0,:])]
      dsi_el  = (dsi_el1[0:len(dsis[:,0])-1] + dsi_el1[1:len(dsis[0,:]),:])/4.0
      vel_el1 = vels[:,0:len(vels[0,:])-1] + vels[:,1:len(vels[0,:])]
      vel_el  = (vel_el1[0:len(vels[:,0])-1] + vel_el1[1:len(vels[0,:]),:])/4.0
      
      # Removing land cells
      nanind = np.argwhere(np.isnan(vol_el))
      for i in range(0,len(nanind)):
        vel_el[nanind[i,0],nanind[i,1]] = np.nan
        din_el[nanind[i,0],nanind[i,1]] = np.nan
        dsi_el[nanind[i,0],nanind[i,1]] = np.nan

      X, Z  = np.meshgrid(x_el[0,:],depth) # Longitude and depth for current section
      Y, Z1 = np.meshgrid(y_el[0,:],depth)
        
      # Total volume transport
      notnan = np.argwhere(~np.isnan(vel_el))
      vel = np.zeros(len(notnan))
      din = np.zeros(len(notnan))
      dsi = np.zeros(len(notnan))
      vol = np.zeros(len(notnan))
      for i in range(0,len(notnan)):
        ii, jj = notnan[i,0], notnan[i,1]
        vel[i] = vel_el[ii,jj]
        din[i] = din_el[ii,jj]
        dsi[i] = dsi_el[ii,jj]
        vol[i] = vol_el[ii,jj]
        
      transp[cnt,0,nsec]    = np.sum(vel*vol)/1.e6
      transpdin[cnt,0,nsec] = np.sum(din*vel*vol)/1.e6
      transpdsi[cnt,0,nsec] = np.sum(dsi*vel*vol)/1.e6
      
      # Positive volume transport
      pos = np.argwhere(vel_el > 0.)
      vel = np.zeros(len(pos))
      din = np.zeros(len(pos))
      dsi = np.zeros(len(pos))
      vol = np.zeros(len(pos))
      for i in range(0,len(pos)):
        ii, jj = pos[i,0], pos[i,1]
        vel[i] = vel_el[ii,jj]
        din[i] = din_el[ii,jj]
        dsi[i] = dsi_el[ii,jj]
        vol[i] = vol_el[ii,jj]
        
      transp[cnt,1,nsec]    = np.sum(vel*vol)/1.e6
      transpdin[cnt,1,nsec] = np.sum(din*vel*vol)/1.e6
      transpdsi[cnt,1,nsec] = np.sum(dsi*vel*vol)/1.e6  
  
      # Negative volume transport    
      neg = np.argwhere(vel_el < 0.)
      vel = np.zeros(len(neg))
      din = np.zeros(len(neg))
      dsi = np.zeros(len(neg))
      vol = np.zeros(len(neg))
      for i in range(0,len(neg)):
        ii, jj = neg[i,0], neg[i,1]
        vel[i] = vel_el[ii,jj]
        din[i] = din_el[ii,jj]
        dsi[i] = dsi_el[ii,jj]
        vol[i] = vol_el[ii,jj]
        
      transp[cnt,2,nsec]    = np.sum(vel*vol)/1.e6
      transpdin[cnt,2,nsec] = np.sum(din*vel*vol)/1.e6 
      transpdsi[cnt,2,nsec] = np.sum(dsi*vel*vol)/1.e6 
    cnt += 1
  # Saving variables
  filename = 'Transports'+str(years[0])
  f = open(filename, 'wb')
  pickle.dump([transp,transpdin,transpdsi], f, 2)  
  f.close() 

months = np.arange(2, 2*num_rec+2, 2)


dpicnt = 200
if dpicnt == 600:
    labsize,titlesize = 10, 16
elif dpicnt == 200:
    labsize,titlesize = 8, 10

labels = 'Jan','Mar','May','Jul','Sep','Nov'

tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.)

almost_black = [(38.,38.,38.)]
r,g,b = almost_black[0]
almost_black[0]=(r / 255., g / 255., b / 255.)
ab = almost_black[0]

fig = plt.figure(num=1,figsize=(9.5,5), dpi=dpicnt, facecolor='w', edgecolor=ab)
fig.subplots_adjust(wspace=0.5,left=0.1,right=0.9)
ax1 = fig.add_subplot(2,2,1) 
ax1.plot(months, transpdin[:,0], '-',color=tableau20[0],label='HIGH')
plt.xlim(1,365)
#plt.ylim(-30,170)
ax1.set_ylabel(r'Transport of DIN [kmol s$^{-1}$]', color=tableau20[0],fontsize=labsize)
for tl in ax1.get_yticklabels():
    tl.set_color(tableau20[0])
for tick in ax1.yaxis.get_major_ticks():
    tick.label.set_fontsize(labsize)       
        
"""ax2 = ax1.twinx()
ax2.plot(months, chl_egc1, '-',color=tableau20[4])
plt.xlim(2,364)
plt.ylim(0,5)
#ax2.set_ylabel('Chl conc. [mmol m$^{-3}$]', color=tableau20[4],rotation=270, labelpad=15,fontsize=labsize)
for tl in ax2.get_yticklabels():
    tl.set_color(tableau20[4])
ax2.tick_params(axis='y',labelsize=labsize)"""

for tl in ax1.get_xticklabels():
    tl.set_color(almost_black[0])
ax1.spines['bottom'].set_color(ab)
ax1.spines['top'].set_color(ab)
ax1.spines['left'].set_color(ab)
ax1.spines['right'].set_color(ab)
ax1.tick_params(axis='x', colors=ab,labelsize=labsize)
ax1.axes.get_xaxis().set_ticklabels([])
ax1.set_xticks([2,59,120,181,243,304])
ax1.set_xticklabels(labels)   

plt.show()   


Loading mesh
The *usepickle = True* and the pickle file (*pickle_mesh*) exists.
 We load the mesh from it.
14.28 seconds process time
Starting calculations
Searching for section


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices